In [3]:
import cv2
import os
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib.request as urllib2

In [4]:
def getmin(p1,p2,p3,p4):
    minx =  min(p1[0],p2[0],p3[0],p4[0])
    miny =  min(p1[1],p2[1],p3[1],p4[1])
    return(minx,miny) 

In [5]:
def getmax(p1,p2,p3,p4):
    maxx =  max(p1[0],p2[0],p3[0],p4[0])
    maxy =  max(p1[1],p2[1],p3[1],p4[1])
    return(maxx,maxy)

In [6]:
def saveparkingspace(newimg,spot):
    dirname = 'Spots'
    resizeimage = cv2.resize(spot,(50,70)) #resizes image to 50 x 70
    cv2.imwrite(os.path.join(dirname, newimg + '.jpg'), resizeimage)

In [7]:
def maskimage(img, topleft,topright,botright,botleft):
    mask = np.zeros(img.shape, dtype=np.uint8)
    roi_corners = np.array([[topleft, topright, botright,botleft]], dtype=np.int32)  # fill the ROI so it doesn't get wiped out when the mask is applied
    channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
    ignore_mask_color = (255,)*channel_count
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)  # apply the mask
    return(masked_image)

In [8]:
def average(image,index):
    sum = 0
    count = 0
    for row in image:
        for col in row:
            if col[index] !=0:
                sum += col[index]
                count = count+1
                
    return float(sum) / count

In [9]:
def countwhite(image):
    im=Image.open( "canny edges/"+ parkingspacelocation +"Edge.bmp" )
    width,height = im.size
    actualpixel = width * height #how many pixels in the entire image?
    count = 0
    for row in image:
        for col in row:
            if col == 255:
                count = count + 1
    ratioofwhitepixels = float(actualpixel) / count
    print ("Number of pixels in image: " ,actualpixel)
    print ("Ratio of white to image: " ,ratioofwhitepixels)
    return count,ratioofwhitepixels

In [10]:
def averagecolors(image):
    avg = [average(image,0) , average(image,1) , average(image, 2)]
    return (avg)

In [11]:
def cannyedgedetection(spotforcanny):
    sigma=0.33
    v = np.median(spotforcanny)

    lower = int(max(0, (1.0 - sigma) * v)) #calulates the lower bound
    upper = int(min(255, (1.0 + sigma) * v)) #calucates the upper bound

    edges = cv2.Canny(spotforcanny,lower,upper) #returns and iamge map

    white = countwhite(edges) #recieves a touple (num whitepixels,ratio of whitepixels to whole image)

    dirname = 'canny edges\\'
    cv2.imwrite(dirname+parkingspacelocation+'Edge.bmp', edges)
    return white

In [12]:
def withinange(avg1, avg2):
    range = 0.1
    blue_diff = abs(avg1[0]-avg2[0])
    green_diff = abs(avg1[1]-avg2[1])
    red_diff = abs(avg1[2]-avg2[2])
    avg_diff = (blue_diff + green_diff + red_diff)/3
    grades = [blue_diff/255.0, green_diff/255.0, red_diff/255.0]
    #print grades
    for grade in grades:
        if grade > range:
            return False
    
    #if grades[0] > range or grades[1] > range or green_diff > range:
    #    return False
    return True

In [13]:
def boxemup(image,topleft,topright,botright,botleft, color):
    line_sz = 2
    diff = 5
    cv2.line(image,(topleft[0] +diff,topleft[1]),(topright[0] -diff,topright[1]),color, line_sz)#1 -> 2
    cv2.line(image,(topleft[0] +diff,topleft[1]),(botleft[0] +diff ,botleft[1]),color, line_sz) #1 -> 3
    cv2.line(image,(topright[0] -diff ,topright[1]),(botright[0] - diff,botright[1]),color, line_sz) #2 -> 4
    cv2.line(image,(botleft[0] +diff,botleft[1]),(botright[0] -diff ,botright[1]),color, line_sz) #3 -> 4
    return image

In [14]:
def drawBoundBox(img, color):
    line_size = 5
    #top horizontal line
    cv2.line(img, (0, 0), (len(img[0]), 0), color, line_size) 
    #left vertical line
    cv2.line(img, (0, 0), (0, len(img)), color, line_size)
    #bottom horizontal line
    cv2.line(img, (0, len(img)), (len(img[0]), len(img)), color, line_size)
    #right vertical line
    cv2.line(img, (len(img[0]), 0), (len(img[0]), len(img)), color, line_size)

In [17]:
def saveImgUrl(url, file_name):
    u = urllib2.urlopen(url)
    infile = open(file_name, 'wb')
    meta = u.info()
    file_size = int(meta.get_all("Content-Length")[0])
    #file_size = int(meta.getheaders("Content-Length")[0])
    print ("Downloading: %s Bytes: %s" % (file_name, file_size))

    file_size_dl = 0
    block_sz = 8192
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break
        file_size_dl += len(buffer)
        infile.write(buffer)
        status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
    status = status + chr(8)*(len(status)+1)
    print (status,infile.close())

with open('UFPR05_105.json') as data_file: #Pick the JSON that defines the parking lot you are working with
    lot_data = json.load(data_file)

with open('UFPR05_emptyData.json') as emptyLotFile: #Use the JSON that contains information about the parking lot when empty
    emptyLot_data = json.load(emptyLotFile)
red_color = (0,0,255) #Globaly defines the color red
green_color = (0,255,0) #Globaly defines the color green
if __name__ == "__main__":

    lot_shots = lot_data['data']['snapshots'] #Images of Parking-Lot
    lot_coords = lot_data['data']['coords'] #coordinates of spots in the  parking-lot

    numImg = 9 #how many images do you want to run over?

    nonchangenumImg = numImg #saves the number of images being used into a static varible
    addup = 0

    sumoverallaverage = 0 #to calculate average correctness of all images ran

#-------------------(SUBJECT TO CHANGE)--------------------------
    numwhiteratiothreshold = 10 #threshold for numwhiteratio algorithm
    numwhiteratiothresholdstart = numwhiteratiothreshold #where to start the threshold
    numwhiteratiothresholdend = 17 #where to end the threshold
    medianthreshold = 70
    whitediffernecethreshold = 90
#-------------------(SUBJECT TO CHANGE)--------------------------

    for img in lot_shots: #loop through the Parking-Lot images
        if numImg < 1 :
            break #Breaks out of the for-loop if there are no more parking lot images
        
        numImg = numImg - 1 #decrement the number images left

#-----------Calls SaveImgUrl Fuction -------------------
        url = img[1]['img'] #Parking-Lot image's URL
        img_name = img[0]+".jpg" #Gives the Parking-Lot image a name
        saveImgUrl(url, "Origin_Images\\"+img_name) 

        image = cv2.imread('Origin_Images\\' + img_name) #reads the image that was just saved
        
        spotColors = [] #Used to save the color for each spot

#----------FOR CALCULATING PERCENT OF CORRECTNESS---------------
        correctspots = 0 #Used to hold how many spots are taken in the Parking-Lot
        percentcorrect = 0 #Used to hold the value of how correct we we were

        for parkingspot in lot_coords: #loops through the Parking-Lot for each individual spot
            decidedfullorempty = 0 #Used to hold the value (0 or 1) 1 for taken  0 for empty
            
            parkingspacelocation = "Space_#_" + str(parkingspot[0]) #Parking space location in the Parking-Lot

#------------------- GETS THE COORDINATES FOR THE SPOT BEING PROCESSED ------------
            topleft = (int(parkingspot[1][3]['x']),int(parkingspot[1][3]['y'])) 
            topright = (int(parkingspot[1][2]['x']),int(parkingspot[1][2]['y']))        
            botleft = (int(parkingspot[1][0]['x']),int(parkingspot[1][0]['y']))
            botright= (int(parkingspot[1][1]['x']),int(parkingspot[1][1]['y']))

#------------------- CODE THAT APPLYS MASK TO PARKING SPACE -----------------------
            masked_image = maskimage(image,topleft,topright,botright,botleft)
            minpoint = getmin(topleft,topright,botleft,botright)
            maxpoint = getmax(topleft,topright,botleft,botright)
            maskedparkingspace = masked_image[minpoint[1]:maxpoint[1], minpoint[0]:maxpoint[0]]

            denoise = cv2.fastNlMeansDenoisingColored(maskedparkingspace,None,20,21,7,31) #gets rid of image noise
            print(parkingspacelocation)
            #print('Gavg',gray_spot_avg)
            avg = averagecolors(denoise) #gets the average of [R,G,B]
            #print('Cavg',avg)
            colorResult = [withinange(emptyLot_data['spots'][parkingspot[0]-1][1], avg)]


            gray_image = cv2.cvtColor(denoise, cv2.COLOR_BGR2GRAY) #convert the denoise'd image to grayscale
            blur = cv2.GaussianBlur(gray_image,(5,5),0) #blur the image

            numberofwhite = cannyedgedetection(blur) #Calls cannyedgedectection funcion, and recives a touple

            whitediffromempty = numberofwhite[0] - emptyLot_data['spots'][parkingspot[0]-1][2] #Differnce of number of white pixels to the spot and it's empty counterpart

            print ("Diff of whitepixels: ", whitediffromempty)
            median = np.median(blur) #gets the median of the blurred image
            print ("Median: " , median)


#-------------BELOW IS WHERE WE APPLY THE ALGORITHMS TO DIFFERENT THRESHOLDS----------------
            if median < medianthreshold: #if yes then the spot is taken
                 drawBoundBox(maskedparkingspace, red_color)
                 spotColors.append([topleft, topright, botleft, botright, red_color])
                 decidedfullorempty = 1
            elif numberofwhite[1] <= numwhiteratiothreshold: #if yes then the spot is taken
                 drawBoundBox(maskedparkingspace, red_color)
                 spotColors.append([topleft, topright, botleft, botright, red_color])
                 decidedfullorempty = 1
            elif whitediffromempty > whitediffernecethreshold: #if yes then the spot is taken
                drawBoundBox(maskedparkingspace, red_color)
                spotColors.append([topleft, topright, botleft, botright, red_color])
                decidedfullorempty = 1
            else:
                drawBoundBox(maskedparkingspace, green_color)
                spotColors.append([topleft, topright, botleft, botright, green_color])
                decidedfullorempty = 0


            if (int(lot_data['data']['snapshots'][addup][1]["lotSnapshot"][str(parkingspot[0])]) == 0):
                print('Actual result from Json: OPEN') #Uses JSON the print the actual status of the spot
            else:  
                print('Actual result from Json: TAKEN') #Uses JSON the print the actual status of the spot

            if (decidedfullorempty == 0):
                print('Our result: OPEN \n') 
            else:  
                print('Our result: TAKEN \n')

            if (int(lot_data['data']['snapshots'][addup][1]["lotSnapshot"][str(parkingspot[0])]) == decidedfullorempty):
                correctspots = correctspots + 1 #if we were correct then incriment the counter

            saveparkingspace(parkingspacelocation,maskedparkingspace) #save the spot

        for spot in spotColors: #loop through the list of spotColors
            image = boxemup(image, spot[0], spot[1], spot[3], spot[2], spot[4]) #applies the color to each spot

        percentcorrect = (float(correctspots) / len(lot_coords)) * 100   
        sumoverallaverage = sumoverallaverage + percentcorrect
        print ("Percentage of correct spots for the current image:", percentcorrect)
        # ------uses linear interpolation the slowly increment the numwhite threshold --------
        numwhiteratiothreshold =  numwhiteratiothreshold + (numwhiteratiothresholdend - numwhiteratiothresholdstart) / 100.0
        addup = addup + 1
        cv2.imwrite('Output_Images\\'+img_name, image)
    print ("\n Overall Percentage of correctness for the experiment:", sumoverallaverage/nonchangenumImg)

Downloading: Origin_Images\1366030801.jpg Bytes: 230796
    230796  [100.00 None
Space_#_1
Number of pixels in image:  21376
Ratio of white to image:  25.089201877934272
Diff of whitepixels:  38
Median:  32.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_2
Number of pixels in image:  15300
Ratio of white to image:  28.544776119402986
Diff of whitepixels:  -145
Median:  76.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_3
Number of pixels in image:  12150
Ratio of white to image:  29.136690647482013
Diff of whitepixels:  -174
Median:  79.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_4
Number of pixels in image:  10414
Ratio of white to image:  23.402247191011234
Diff of whitepixels:  -125
Median:  79.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_5
Number of pixels in image:  7424
Ratio of white to image:  18.240786240786242
Diff of whitepixels:  -6
Median:  84.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_6
Number of pixels 

Number of pixels in image:  7345
Ratio of white to image:  17.241784037558684
Diff of whitepixels:  -147
Median:  83.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_7
Number of pixels in image:  5885
Ratio of white to image:  10.62274368231047
Diff of whitepixels:  142
Median:  65.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_8
Number of pixels in image:  5432
Ratio of white to image:  14.923076923076923
Diff of whitepixels:  12
Median:  85.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_9
Number of pixels in image:  4232
Ratio of white to image:  9.728735632183907
Diff of whitepixels:  165
Median:  71.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_10
Number of pixels in image:  3956
Ratio of white to image:  8.713656387665198
Diff of whitepixels:  164
Median:  117.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_11
Number of pixels in image:  12420
Ratio of white to image:  12.188420019627085
Diff of whitepixels:  547
Medi

Space_#_12
Number of pixels in image:  12496
Ratio of white to image:  15.091787439613526
Diff of whitepixels:  346
Median:  51.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_13
Number of pixels in image:  9438
Ratio of white to image:  21.45
Diff of whitepixels:  -25
Median:  94.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_14
Number of pixels in image:  9100
Ratio of white to image:  21.162790697674417
Diff of whitepixels:  -73
Median:  94.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_15
Number of pixels in image:  7442
Ratio of white to image:  25.05723905723906
Diff of whitepixels:  -120
Median:  103.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_16
Number of pixels in image:  6728
Ratio of white to image:  23.690140845070424
Diff of whitepixels:  -79
Median:  97.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_17
Number of pixels in image:  4851
Ratio of white to image:  19.098425196850393
Diff of whitepixels:  -103
Med

Space_#_18
Number of pixels in image:  6105
Ratio of white to image:  11.18131868131868
Diff of whitepixels:  116
Median:  44.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_19
Number of pixels in image:  3560
Ratio of white to image:  10.318840579710145
Diff of whitepixels:  70
Median:  48.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_20
Number of pixels in image:  3649
Ratio of white to image:  7.830472103004292
Diff of whitepixels:  135
Median:  81.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_21
Number of pixels in image:  7840
Ratio of white to image:  15.10597302504817
Diff of whitepixels:  263
Median:  65.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_22
Number of pixels in image:  6400
Ratio of white to image:  17.97752808988764
Diff of whitepixels:  61
Median:  62.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_23
Number of pixels in image:  6552
Ratio of white to image:  22.989473684210527
Diff of whitepixe

Space_#_25
Number of pixels in image:  5644
Ratio of white to image:  14.288607594936709
Diff of whitepixels:  86
Median:  88.0
Actual result from Json: TAKEN
Our result: OPEN 

Space_#_26
Number of pixels in image:  4896
Ratio of white to image:  19.821862348178136
Diff of whitepixels:  -46
Median:  99.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_27
Number of pixels in image:  4425
Ratio of white to image:  19.407894736842106
Diff of whitepixels:  -11
Median:  96.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_28
Number of pixels in image:  4575
Ratio of white to image:  13.905775075987842
Diff of whitepixels:  -11
Median:  84.0
Actual result from Json: TAKEN
Our result: OPEN 

Space_#_29
Number of pixels in image:  4958
Ratio of white to image:  15.942122186495176
Diff of whitepixels:  -22
Median:  90.0
Actual result from Json: TAKEN
Our result: OPEN 

Space_#_30
Number of pixels in image:  8470
Ratio of white to image:  12.048364153627311
Diff of whitepixels

Number of pixels in image:  6760
Ratio of white to image:  17.512953367875646
Diff of whitepixels:  -74
Median:  73.0
Actual result from Json: TAKEN
Our result: OPEN 

Space_#_32
Number of pixels in image:  6215
Ratio of white to image:  10.623931623931623
Diff of whitepixels:  280
Median:  90.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_33
Number of pixels in image:  5151
Ratio of white to image:  21.110655737704917
Diff of whitepixels:  -49
Median:  95.0
Actual result from Json: OPEN
Our result: OPEN 

Space_#_34
Number of pixels in image:  3960
Ratio of white to image:  16.431535269709542
Diff of whitepixels:  -31
Median:  81.0
Actual result from Json: TAKEN
Our result: OPEN 

Space_#_35
Number of pixels in image:  4656
Ratio of white to image:  9.023255813953488
Diff of whitepixels:  214
Median:  105.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_36
Number of pixels in image:  3239
Ratio of white to image:  8.825613079019073
Diff of whitepixels:  226
M

Space_#_37
Number of pixels in image:  3649
Ratio of white to image:  10.669590643274853
Diff of whitepixels:  181
Median:  91.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_38
Number of pixels in image:  4071
Ratio of white to image:  16.02755905511811
Diff of whitepixels:  118
Median:  79.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_39
Number of pixels in image:  3477
Ratio of white to image:  15.522321428571429
Diff of whitepixels:  104
Median:  82.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_40
Number of pixels in image:  3520
Ratio of white to image:  17.170731707317074
Diff of whitepixels:  22
Median:  86.0
Actual result from Json: TAKEN
Our result: OPEN 

Percentage of correct spots for the current image: 95.0
Downloading: Origin_Images\1366032901.jpg Bytes: 254618
    254618  [100.00 None
Space_#_1
Number of pixels in image:  21376
Ratio of white to image:  23.20955483170467
Diff of whitepixels:  107
Median:  31.0
Actual result from

Space_#_2
Number of pixels in image:  15300
Ratio of white to image:  16.831683168316832
Diff of whitepixels:  228
Median:  36.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_3
Number of pixels in image:  12150
Ratio of white to image:  20.182724252491695
Diff of whitepixels:  11
Median:  41.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_4
Number of pixels in image:  10414
Ratio of white to image:  10.626530612244897
Diff of whitepixels:  410
Median:  52.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_5
Number of pixels in image:  7424
Ratio of white to image:  16.174291938997822
Diff of whitepixels:  46
Median:  44.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_6
Number of pixels in image:  7345
Ratio of white to image:  12.180762852404644
Diff of whitepixels:  30
Median:  63.0
Actual result from Json: TAKEN
Our result: TAKEN 

Space_#_7
Number of pixels in image:  5885
Ratio of white to image:  11.699801192842942
Diff of whitepixe